In [7]:
import json
import requests
import pandas as pd
import time
import datetime

In [157]:
pd.set_option('display.max_rows', 200)

In [119]:
lol_api_key = 'RGAPI-2eef7b6a-ea60-4cb0-88dc-21d381005c93'

In [127]:
URLS = {'challenge' : 'https://kr.api.riotgames.com/lol/league/v3/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=',
        'master' : 'https://kr.api.riotgames.com/lol/league/v3/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=',
        'summoners' : 'https://kr.api.riotgames.com/lol/summoner/v3/summoners/',
        'matches' : 'https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/',
        'match_detail' : 'https://kr.api.riotgames.com/lol/match/v3/matches/',
        'champions': 'https://kr.api.riotgames.com/lol/static-data/v3/champions?locale=ko_KR&dataById=false&api_key='
       }

# Analysis direction
### 분석대상
- 특정 게임 버전에서 (7.21) challenge league player가 한명이라도 속한 경기.
- Note: https://docs.google.com/spreadsheets/d/1kd6EdIA9FRWOoRbpY_htll4ijSPyrKc9fOhBYIEViK0/edit#gid=853537432

# Ben & pick order check

같은 팀 벤 중복은 불가능  
  
streamerName = '해물파전'  
1.  
accountId: 2663347  
gameId: 2994667322  
https://www.youtube.com/watch?v=qs43TtFqwRk&t=394s

2.  
accountId: 2663347  
gameId: 2994630626  
https://www.youtube.com/watch?v=y62J44JccSU&t=9s  



In [128]:
url = requests.get(URLS['champions'] + lol_api_key)
print(url.status_code)
champions_info = url.json()

In [143]:
df_champions = pd.DataFrame()
for name in champions_info['data'].keys():
    df_champions = df_champions.append([[champions_info['data'][name]['key'], champions_info['data'][name]['id']]])
df_champions.reset_index(inplace=True)
del df_champions['index']
df_champions.columns = ['key','id']

In [158]:
df_champions

,key,id
0,Jax,24
1,Sona,37
2,Tristana,18
3,Varus,110
4,Fiora,114
5,Singed,27
6,TahmKench,223
7,Leblanc,7
8,Thresh,412
9,Karma,43


# Data collection

### Summoner target

In [31]:
# challenge league 선수들 정보 불러옴
url = requests.get(URLS['challenge']+lol_api_key)
print(url.status_code)
challenge = url.json()

200


In [32]:
# master league 선수들 정보 불러옴 / 일단 challenge league선수만으로 한다.
# url = requests.get(URLS['master']+lol_api_key)
# print(url.status_code)
# master = url.json()

In [33]:
# match를 모을 target을 challenge, master 리그 summerner들로 잡음 / 일단 challenge league선수만으로 한다.
# df_target_origin = pd.DataFrame(challenge['entries']).append(pd.DataFrame(master['entries']), ignore_index=True)
df_target_origin = pd.DataFrame(challenge['entries'])
df_target_origin[df_target_origin.playerOrTeamName == 'Afreeca Kuro']

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins
166,False,False,False,645,344,1222794,Afreeca Kuro,I,True,413


In [39]:
# X-App-Rate-Limit에 맞춰 호출함. 
# status_code=429 는 api호출 한계 초과임
#     "X-App-Rate-Limit": "100:120,20:1",
#     "X-Method-Rate-Limit": "2000:60",
summoners = []
# counts = 0
for i in df_target_origin.playerOrTeamId:
    time.sleep(0.9)
#     counts += 1
#     print(counts)
    url = requests.get(URLS['summoners'] + i + '?api_key=' + lol_api_key)
    while url.status_code == 429:
        time.sleep(1)
        url = requests.get(URLS['summoners'] + i + '?api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
    else:
        summoners.append(url.json())

pd.DataFrame(summoners).to_csv('summonors.csv', index = False)

In [42]:
df_summonors = pd.read_csv('summonors.csv')[['accountId','id']]

In [43]:
# 이렇게 안하면 merge시 조인이 안됨 <- 데이터 타입이 달라서 생기는 문제인데 꼭 이렇게 해야 할까?
df_target_origin['playerOrTeamId'] = df_target_origin['playerOrTeamId'].astype(int)

In [44]:
df_target = pd.merge(df_target_origin, df_summonors, left_on='playerOrTeamId', right_on = 'id', how='inner')

In [45]:
df_target[df_target.playerOrTeamName == 'Afreeca Kuro']

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins,accountId,id
166,False,False,False,645,344,1222794,Afreeca Kuro,I,True,413,232934,1222794


### Match

In [48]:
# version별 분석을 하고 싶어서 기준일을 timestamp로 지정한다. + 9 시간을 고려한다.
# 7.21 시작일 2017-10-25 한국시간기준
timestamp_begin = int(datetime.datetime.timestamp(datetime.datetime(2017,10,26,0,0,0))) * 1000
timestamp_end = int(datetime.datetime.timestamp(datetime.datetime(2017,10,30,0,0,0))) * 1000

print(timestamp_begin, timestamp_end)

1508943600000 1509289200000


In [49]:
# match 를 선수별로 모은다.
# 선수당 max 100개만 모은다. 라이엇 정책이 1회 조회에 100개. / 100개 이상으로도 돌릴 수 있지만 이번엔 이정도까지만.
# "X-Method-Rate-Limit-Count": "1:10"
# "X-App-Rate-Limit": "100:120,20:1"
# "status_code": 404, "message": "Not found" 
# - challenge league summoner인데 이번 버전에서 플레이 하지 않아서 찾을 수 있는 값이 없을 수 있음.

matches = dict()
accountId_exeptions = []
# counts = 0
for i in df_target.accountId:
    time.sleep(0.8)
#     counts += 1
#     print(counts, i)
    url = requests.get(URLS['matches'] + str(i) + '?endTime=' + str(timestamp_end) + '&beginTime=' + \
                       str(timestamp_begin) + '&api_key=' + lol_api_key)
    while url.status_code == 429:
        time.sleep(0.8)
        url = requests.get(URLS['matches'] + str(i) + '?endTime=' + str(timestamp_end) + '&beginTime=' + \
                           str(timestamp_begin) + '&api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
        accountId_exeptions.append([i, url.status_code])
    else:
        matches[str(i)] = url.json()

with open('matches.json', 'w') as js:
     json.dump(matches, js)

8053322 404
206487982 404
6358198 404
205065873 404
6871165 404
665129 404
4741330 404
203881305 404
232934 404
200222990 404
4267501 404
1110666 404


### gameId

- challenge league summoner가 한명이라도 속한 경기를 추출한다.

In [50]:
with open('matches.json', 'r') as js:
     mataches_json = json.load(js)

In [58]:
mataches_json['2533789'].keys()

dict_keys(['totalGames', 'endIndex', 'matches', 'startIndex'])

In [87]:
gameIds = []
for accountId in mataches_json:
    for i in range(mataches_json[accountId]['endIndex']):
        gameIds.append(mataches_json[accountId]['matches'][i]['gameId'])
unique_gameIds = list(set(gameIds))

### gameId_detail

In [116]:
# match 세부사항을 gameId별로 모은다.
# "X-App-Rate-Limit": "100:120,20:1"
# "X-Method-Rate-Limit": "500:10"
# len(unique_gameIds) = 985
match_details = dict()
gameID_exeptions = []
# counts = 0
for unique_gameId in unique_gameIds:
    time.sleep(0.4)
#     counts += 1
#     print(counts, i)
    url = requests.get(URLS['match_detail'] + str(unique_gameId) + '?api_key=' + lol_api_key)
    while url.status_code == 429:
        time.sleep(0.8)
        url = requests.get(URLS['match_detail'] + str(unique_gameId) + '?api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
        gameID_exeptions.append([i, url.status_code])
    else:
        match_details[str(unique_gameId)] = url.json()

with open('match_details.json', 'w') as js:
     json.dump(match_details, js)

# Data cleansing

In [117]:
with open('match_details.json', 'r') as js:
     match_details_json = json.load(js)

In [118]:
match_details_json['2994319360']

{'gameCreation': 1509091110557,
 'gameDuration': 1847,
 'gameId': 2994319360,
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'gameVersion': '7.21.206.6866',
 'mapId': 11,
 'participantIdentities': [{'participantId': 1,
   'player': {'accountId': 3917086,
    'currentAccountId': 3917086,
    'currentPlatformId': 'KR',
    'matchHistoryUri': '/v1/stats/player_history/KR/3917086',
    'platformId': 'KR',
    'profileIcon': 7,
    'summonerId': 4886229,
    'summonerName': '귤         1'}},
  {'participantId': 2,
   'player': {'accountId': 206560208,
    'currentAccountId': 206560208,
    'currentPlatformId': 'KR',
    'matchHistoryUri': '/v1/stats/player_history/KR/206560208',
    'platformId': 'KR',
    'profileIcon': 3163,
    'summonerId': 39050997,
    'summonerName': '종학깅'}},
  {'participantId': 3,
   'player': {'accountId': 208752837,
    'currentAccountId': 208752837,
    'currentPlatformId': 'KR',
    'matchHistoryUri': '/v1/stats/player_history/KR/208752837',
    'platformI

# 참고자료

In [ ]:
API_KEY = os.environ['LOL_API_KEY']
URLS = {
        'NAME_TO_ID' : 'https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-name/{name}',
        'ID_TO_NAME' : 'https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-account/{id}',
        'MATCHES_BY_ID' : 'https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/{id}?season=8',
        'RECENT_MATCHES_BY_ID' : 'https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/{id}/recent',
        'MATCH_INFO': 'https://na1.api.riotgames.com/lol/match/v3/matches/{id}', 
        'RECENT_TEN' : 'https://na.api.riotgames.com/api/lol/NA/v1.3/game/by-summoner/{summonerId}/recent',
        }
HEADERS = {'X-Riot-Token' : API_KEY}

CHAMP_BY_ID = {}
with open('data/champ_by_id.json') as f:
    CHAMP_BY_ID = json.load(f)

CHAMP_BY_NAME = {}
with open('data/champ_by_name.json') as f:
    CHAMP_BY_NAME = json.load(f)


print(CHAMP_BY_ID['data']['236'])

In [ ]:
from apiKeys import API_KEY
import requests
import sys
import pprint

USERNAME = sys.argv[1]
BASE_URL = "https://na1.api.riotgames.com/lol/"
SUMMONER_ID_URL = BASE_URL + "summoner/v3/summoners/by-name/"
MASTERY_URL = BASE_URL + "platform/v3/masteries/by-summoner/"

r = requests.get(SUMMONER_ID_URL + USERNAME + "?api_key=" + API_KEY)
print r.status_code
data = r.json()
print data
print data["name"]
print data["id"]
summoner_id = data["id"]
r = requests.get(MASTERY_URL + str(summoner_id) + "?api_key=" + API_KEY)
pprint.pprint(r.json())
